# Data Collection

Collecting data for the both the initial goal of clustering Redditors but also for the next step of the process, which is predicting game sales based on Reddit Submissions, Comments, and Users.

**Steps**
1. Create a list of games released on Steam that we are interested in.
2. Get release dates listed from Steam API.
3. Get submissisions about game on reddit, pre and post launch within a 5 day range. (5 is arbitrary)
3. Get comments from each submission.
4. Get reddit user information of authors of submissions and comments.

## Table of Contents
- [Steam](#Steam-Game-List-&-Data)
- [Reddit](#Reddit-Data) 

**Imports**

In [1]:
import configparser
import datetime as dt
import time

import dateparser
import numpy as np
import pandas as pd
import praw
import requests

from prawcore.exceptions import (Forbidden as PrawForbiddenError,
                                 NotFound as PrawNotFound)

from aws_mysql import MySqlConn

**Functions**

In [2]:
# TODO
def retry():
    """
    Retry api call on failure.
    """
    pass

# Steam ---------------------------------------------------------------------
def get_steam_app_id(name, app_ids):
    """
    Gets game Steam app id.
    :param: String, steam game name.
    :param app_ids: List, of dictionaries with Steam app name and id (From 
    Steam API)
    :return: String.
    """
    for app in app_ids:
        # TODO: implement fuzzy matching here
        if name.lower() == app['name'].lower():
            return str(app['appid'])

def get_game_data(steam_app_id, url):
    """
    Gets steam data of game from Steam API.
    :param steam_app_id: Int, Steam app id.
    :param url: Steam app url for getting app data.
    :return: List of dictionaries.
    """
    res = requests.get(url + steam_app_id)  
    if res.status_code != 200:
        print(f'Warning: {name} | {res.status_code}')
    return res.json()[steam_app_id]['data']

def get_dates(release_date, spread=5):
    """
    Calculates the initial and end days of range spread, before and
    after the release date, does not include the release date.
    :param release_date: DateTime, release date of game.
    :param spread: Number of days for the date range.
    :return: Tuple of four dates (epoch time, int) in order:
        Pre Launch Start Date, 
        Pre Launch End Date
        Post Launch Start Date
        Post Launch End Date
    """
    # pushshift api expects date param to be an int
    return (
        int((release_date - dt.timedelta(days=spread)).timestamp()),
        int((release_date - dt.timedelta(seconds=1)).timestamp()),
        int((release_date + dt.timedelta(days=1)).timestamp()),
        int((release_date + dt.timedelta(days=spread + 1)).timestamp())
    )

# Reddit --------------------------------------------------------------------
def get_submissions(search_term, start_epoch, end_epoch, sleep=1):
    """
    Uses Pushshift Reddit API to get all submissions for a search term within
    the provided date range.
    :param search_term: String used as search term in API call.
    :param start_epoch: Beginning date, int, to look for submissions.
    :param end_epoch: End date, int, to look for submissions.
    :param sleep: Time(seconds) to wait before running next API call. Defaults
    to 1 second.
    :return: Pandas DataFrame
    """
    params = {
        # q is more exhaustive than using title and selftext
        'q': search_term,  
#         'title': search_term,
#         'selftext': search_term,
        'after': start_epoch,
        'before': end_epoch,
        'size': 100  # 100 is API limit
    }
    
    prev_size = -1  # to check if there are results
    all_submissions = []
    while len(all_submissions) > prev_size:
        prev_size = len(all_submissions)
        res = requests.get(
            'https://api.pushshift.io/reddit/search/submission',
            params
        )
        
        if res.status_code == 200:
            submissions = res.json()['data']
            # TODO:
            # depending on what sbumissions returns when empty, can replace len(all_sub) > above
            if submissions:  
                all_submissions += submissions
                # do not want duplicate data
                params['after'] = max([submission['created_utc'] 
                                       for submission in submissions])
        time.sleep(sleep)
    df_ = pd.DataFrame(all_submissions)
    df_['search_term'] = search_term
    return df_

def get_comments(submission_id):
    """
    Gets all the comments of a submission and returns comment data and author 
    objects, (Redditor Instances).
    :param submission_id: String, id of reddit submission.
    :return: Tuple (List of data, List of Authors as Redditor Objects)
    """
    comments = []
    authors = []
    # https://praw.readthedocs.io/en/latest/tutorials/comments.html
    # writing over id but following documentation
    submission = reddit.submission(id=submission_id)  
    submission.comments.replace_more(limit=None)  # grab all comments
    for comment in submission.comments.list():
        print(len(comments))
        try:
            comments.append((
                comment.id,
                comment.author.id if comment.author else None,
                comment.body,
                comment.created_utc,
                comment.distinguished,
                comment.edited,
                comment.is_submitter,
                comment.link_id,
                comment.parent_id, 
                comment.score,
                comment.stickied,
                comment.subreddit_id
            ))

            authors.append(comment.author)
        except AttributeError as e:  # attribute error is from comments.author.id, prob if author is deleted
            comments.append((
                comment.id,
                None,
                comment.body,
                comment.created_utc,
                comment.distinguished,
                comment.edited,
                comment.is_submitter,
                comment.link_id,
                comment.parent_id, 
                comment.score,
                comment.stickied,
                comment.subreddit_id
            ))
        except (PrawForbiddenError, PrawNotFound):
            pass
            
    return comments, authors
    
def save_comments_and_authors(submission_ids):
    """
    Saves comment and author data into AWS rds.
    :param submission_ids: List/array of reddit submission ids.
    :return: None
    """
    db = MySqlConn(ENDPOINT, USER, PASSWORD, DBNAME)
    
    for submission_id in submission_ids:
        try:  # catch private submissions that may cause forbidden errors
            comments, authors = get_comments(submission_id)
            comments_sql = """
                INSERT INTO comments (CommentID, 
                                      AuthorID, 
                                      Body, 
                                      CreatedUTC, 
                                      Distinguished, 
                                      Edited, 
                                      IsSubmitter, 
                                      LinkID, 
                                      ParentID, 
                                      Score, 
                                      Stickied, 
                                      SubredditID) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """
            db.batch_insert(comments_sql, comments)
            comments = []

            users = {(author.id, 
                      author.comment_karma, 
                      author.created_utc, 
                      author.has_verified_email, 
                      author.is_employee, 
                      author.is_mod, 
                      author.is_gold, 
                      author.link_karma,
                      author.name) for author in authors if author}

            users_sql = """
                INSERT INTO users (UserID, 
                                   CommentKarma, 
                                   CreatedUTC, 
                                   VerifiedEmail, 
                                   IsEmployee, 
                                   IsMod, 
                                   IsGold, 
                                   LinkKarma, 
                                   Name) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    UserID = UserID;
            """
            db.batch_insert(users_sql, users)
            authors = []

            print(f'Last updated submission id: ', submission_id)
            time.sleep(1)
        except PrawForbiddenError:
            pass

### Configuration

Pull in all necessary credentials to access AWS RDS and Praw.   
`config.ini` layout:  

```
[rds]  
endpoint: endpoint  
user: username
password: password
dbname: databasename


[praw]  
client_id: clientid  
client_secret: clientsecret  
user_agent: useragent  
```

In [3]:
config = config = configparser.ConfigParser()
config.read('./config.ini')

ENDPOINT = config['rds']['endpoint']
USER     = config['rds']['user']
PASSWORD = config['rds']['password']
DBNAME   = config['rds']['dbname']

CLIENTID     = config['praw']['client_id']
CLIENTSECRET = config['praw']['client_secret']
USERAGENT    = config['praw']['user_agent']

## Steam Game List & Data

List of games that are of interest. Games were chosen based on the following factors:
* Recency, post 2016
* Games without early access (release dates do not represent when game became available for play)
* Relatively successful games/well known games

In [4]:
# list of games for which to pull data
games = [
    "Fallout 4",
    "Cyberpunk 2077",
    "Stardew Valley",
    "Monster Hunter: World",
    "Sid Meier's Civilization IV",
    "Sid Meier's Civilization V",
    "Sid Meier's Civilization VI",
    "XCOM 2",
    "The Witcher 3: Wild Hunt",
    "Valheim",
    "DARK SOULS™ II",
    "DARK SOULS™ III",
    "Bioshock Infinite",
    "Total War: WARHAMMER",
    "Total War: WARHAMMER II",
    "Total War: THREE KINGDOMS",
    "Tropico 6",
    "Warhammer: Vermintide 2",
    "Red Dead Redemption 2",
    "Hades",
    "Bastion",
    "They are Billions",
    "Portal 2",
    "Doom",
    "Doom Eternal"
]

# Steam app list url and app url for game data
steam_app_list_url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
steam_app_url = 'https://store.steampowered.com/api/appdetails?appids='

# get list of Steam games and app ids
res = requests.get(steam_app_list_url)
steam_apps = res.json()['applist']['apps']

# get a list of steam app ids
steam_app_ids = [get_steam_app_id(game, steam_apps) for game in games]

# get Steam game data
game_data = [get_game_data(app_id, steam_app_url) 
             for app_id in steam_app_ids if app_id]

# want data in DataFrame, might use later
games_df = pd.DataFrame(game_data)
games_df.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,movies,recommendations,achievements,release_date,support_info,background,content_descriptors,demos,drm_notice,ext_user_account_notice
0,game,Fallout 4,377160,17,False,full,"[540810, 404090, 435881, 480631, 480630, 49065...","<h1>Review Highlights</h1><p><img src=""https:/...","Bethesda Game Studios, the award-winning creat...","Bethesda Game Studios, the award-winning creat...",...,"[{'id': 256658080, 'name': 'Fallout 4 Launch T...",{'total': 149509},"{'total': 84, 'highlighted': [{'name': 'War Ne...","{'coming_soon': False, 'date': 'Nov 9, 2015'}","{'url': 'http://help.bethsoft.com/', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN
1,game,Cyberpunk 2077,1091500,17,False,NaN,NaN,<h1>Check out other games from CD PROJEKT RED<...,"<img src=""https://cdn.cloudflare.steamstatic.c...","Cyberpunk 2077 is an open-world, action-advent...",...,"[{'id': 256810255, 'name': '022_CP_Gameplay_Tr...",{'total': 371825},"{'total': 44, 'highlighted': [{'name': 'The Fo...","{'coming_soon': False, 'date': 'Dec 9, 2020'}","{'url': 'http://en.cdprojektred.com/support', ...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [1, 2, 5], 'notes': 'Cyberpunk 2077 co...",NaN,NaN,NaN
2,game,Stardew Valley,413150,0,False,full,[440820],Stardew Valley is an open-ended country-life R...,Stardew Valley is an open-ended country-life R...,You've inherited your grandfather's old farm p...,...,"[{'id': 256660296, 'name': 'Stardew Valley Tra...",{'total': 309442},"{'total': 40, 'highlighted': [{'name': 'Greenh...","{'coming_soon': False, 'date': 'Feb 26, 2016'}","{'url': '', 'email': 'support@stardewvalley.net'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN
3,game,Monster Hunter: World,582010,0,False,NaN,"[1118010, 912540, 960781, 896580, 763695, 7636...","<h1>Featured DLC</h1><p><a href=""https://store...",Welcome to a new world! Take on the role of a ...,Welcome to a new world! In Monster Hunter: Wor...,...,"[{'id': 256769025, 'name': '191128_防衛隊派生武器_ハング...",{'total': 196627},"{'total': 100, 'highlighted': [{'name': 'Conqu...","{'coming_soon': False, 'date': '9 Aug, 2018'}","{'url': 'http://www.capcom.com/mhwsupport/', '...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN
4,game,Sid Meier's Civilization® IV,3900,0,False,NaN,NaN,With over 6 million units sold and unprecedent...,With over 6 million units sold and unprecedent...,With over 6 million units sold and unprecedent...,...,NaN,{'total': 1733},NaN,"{'coming_soon': False, 'date': 'Oct 25, 2006'}","{'url': 'http://support.2kgames.com/', 'email'...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN


### Pre & Post Date Ranges

Need pre and post launch start and end dates as parameters for PushShift API to get submissions in a date range.

In [5]:
# api returns dictionary release_date {'coming_soon': bool, 'date': str}
# need to use dateparser as dates come in different languages
games_df['release_date'] = games_df['release_date'].map(lambda x: 
                                                        dateparser.\
                                                        parse(x['date']))
games_df['release_date'] = pd.to_datetime(games_df['release_date'])

# get the pre and post launch date ranges
games_df['launch_dates'] = games_df['release_date'].apply(get_dates)

# take ranges and put each date into it's own column *Appendix Code Ref 1
new_cols = [
    'pre_launch_start_date', 
    'pre_launch_end_date', 
    'post_launch_start_date',
    'post_launch_end_date'
]

for i, col in enumerate(new_cols):
    games_df[col] = games_df['launch_dates'].apply(lambda dates: dates[i])

games_df = games_df.drop('launch_dates', axis=1)

In [12]:
# not yet used in modeling process
# # save to csv as game data has a lot of information
# games_df.to_csv('./data/games.csv', index=False)

## Reddit Data

In [7]:
# connection to Redit API through praw
reddit = praw.Reddit(
    client_id=CLIENTID,
    client_secret=CLIENTSECRET,
    user_agent=USERAGENT
)

Version 7.0.0 of praw is outdated. Version 7.3.0 was released Thursday June 17, 2021.


### Submissions

For each game, get all Reddit submissions that were posted within the desired date range. Then save to a csv file that will be cleaned later.

In [8]:
game_submissions = []
for game in games_df[['name',
                      'pre_launch_start_date',
                      'pre_launch_end_date',
                      'post_launch_start_date',
                      'post_launch_end_date']].values:
    # need two separate calls (pre and post release date) as release date is
    # not included
    df_pre = get_submissions(game[0], game[1], game[2])
    df_pre['period'] = 'pre launch'
    game_submissions.append(df_pre)
    
    df_post = get_submissions(game[0], game[3], game[4])
    df_post['period'] = 'post launch'
    game_submissions.append(df_post)
    
submissions_df = pd.concat(game_submissions, ignore_index=True)
print(submissions_df.shape)
submissions_df.head()

(86657, 122)


,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,created_utc,domain,edited,full_link,gilded,...,brand_safe,mod_reports,user_reports,updated_utc,category,steward_reports,removed_by,event_end,event_is_live,event_start
0,watisdisanywais,1.417940e+09,enclave,Enclave4Lyfe,t2_juza0,1.446595e+09,self.Fallout,1.446596e+09,https://www.reddit.com/r/Fallout/comments/3rf9...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fatyg5,1.380310e+09,text-desktop color-pcmr icon-steam,FX-8320@4.0GHz | R9 290 | 8GB,t2_db1a5,1.446595e+09,self.pcmasterrace,NaN,https://www.reddit.com/r/pcmasterrace/comments...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rodolfo619,1.358613e+09,None,None,t2_aabya,1.446595e+09,self.Fallout,NaN,https://www.reddit.com/r/Fallout/comments/3rfa...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INBREEDCLOWN,1.445405e+09,None,None,t2_rc9lq,1.446596e+09,self.fo4,NaN,https://www.reddit.com/r/fo4/comments/3rfai7/m...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pyjamapants8,1.391557e+09,None,None,t2_f4jkh,1.446596e+09,youtube.com,NaN,https://www.reddit.com/r/VGCovers/comments/3rf...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# check how many submissions per game
submissions_df['search_term'].value_counts()

Cyberpunk 2077                  27025
Fallout 4                       24446
DOOM Eternal                     8452
DOOM                             5679
Portal 2                         4533
BioShock Infinite                3807
XCOM® 2                          2401
Monster Hunter: World            1776
DARK SOULS™ III                  1226
Red Dead Redemption 2            1201
The Witcher® 3: Wild Hunt        1047
Total War: WARHAMMER              938
Hades                             847
They Are Billions                 543
Warhammer: Vermintide 2           480
Stardew Valley                    456
DARK SOULS™ II                    425
Total War: THREE KINGDOMS         423
Tropico 6                         337
Total War: WARHAMMER II           326
Bastion                           190
Valheim                            77
Sid Meier’s Civilization® VI       16
Sid Meier's Civilization® V         6
Name: search_term, dtype: int64

In [10]:
submissions_df.to_csv('./data/submissions.csv', index=False)

### Comments & Authors

Getting every comment and every author for each comment in every submission and pushing them to an AWS rds instance.

In [11]:
# when rerunning `save_comments_and_authors() read in csv to prevent pending 
# time on getting submissions
# submissions_df = pd.read_csv('./data/submissions.csv')

**Start Data Collection, No Previous Data**

In [ ]:
save_comments_and_authors(submissions_df['id'].values)

**Continue Data Collection, Previous Data Saved in DB**

In [ ]:
# if there is error during data collection, get the next index after the
# last successful API call & write and restart data collection
last_id = '3s7j5u'
last_index = submissions_df[submissions_df['id'] == last_id].index[0] + 1
print(last_index)

save_comments_and_authors(submissions_df['id'][last_index:].values)